# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 10:54AM,240840,19,DEX0007093,Dexcel Pharma Technologies Ltd. DBA WDSrx,Executing
1,Jun 30 2022 10:54AM,240840,19,DEX0007094,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 30 2022 10:54AM,240840,19,DEX0007095,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 30 2022 10:47AM,240838,10,0085817259,ISDIN Corporation,Released
4,Jun 30 2022 10:47AM,240838,10,0085815092,ISDIN Corporation,Released
5,Jun 30 2022 10:47AM,240838,10,0085815096,ISDIN Corporation,Released
6,Jun 30 2022 10:47AM,240838,10,0085815093,ISDIN Corporation,Released
7,Jun 30 2022 10:47AM,240838,10,0085817089,ISDIN Corporation,Released
8,Jun 30 2022 10:47AM,240838,10,0085817112,ISDIN Corporation,Released
9,Jun 30 2022 10:47AM,240838,10,0085817114,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,240836,Executing,1
35,240836,Released,1
36,240838,Released,21
37,240840,Executing,1
38,240840,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240834,NaN,1.0
240835,NaN,1.0
240836,1.0,1.0
240838,NaN,21.0
240840,1.0,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240750,25.0,1.0
240766,0.0,1.0
240768,0.0,1.0
240774,1.0,0.0
240776,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240750,25,1
240766,0,1
240768,0,1
240774,1,0
240776,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,0,1
2,240768,0,1
3,240774,1,0
4,240776,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240750,25,1
1,240766,,1
2,240768,,1
3,240774,1,
4,240776,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation
24,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc."
26,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation
27,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation
33,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc."
34,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation
71,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC"
72,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.
73,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,2
1,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,,21
2,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",1,1
3,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,,1
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,,6
5,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",,1
6,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,,37
7,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",,1
8,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,,1
9,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,1
1,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21,
2,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",1,1
3,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1,
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,
5,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1,
6,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,37,
7,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1,
8,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1,
9,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,1
1,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21,
2,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",1,1
3,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1,
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,1.0
1,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21.0,NaN
2,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",1.0,1.0
3,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1.0,NaN
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,1.0
1,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21.0,0.0
2,Jun 30 2022 10:45AM,240836,10,"Methapharm, Inc.",1.0,1.0
3,Jun 30 2022 10:43AM,240835,10,ISDIN Corporation,1.0,0.0
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2649012,142.0,37.0
12,481619,0.0,3.0
16,722424,5.0,4.0
17,240768,1.0,0.0
18,240813,1.0,0.0
19,1204103,31.0,31.0
20,1444761,23.0,50.0
21,1203910,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2649012,142.0,37.0
1,12,481619,0.0,3.0
2,16,722424,5.0,4.0
3,17,240768,1.0,0.0
4,18,240813,1.0,0.0
5,19,1204103,31.0,31.0
6,20,1444761,23.0,50.0
7,21,1203910,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,142.0,37.0
1,12,0.0,3.0
2,16,5.0,4.0
3,17,1.0,0.0
4,18,1.0,0.0
5,19,31.0,31.0
6,20,23.0,50.0
7,21,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,12,Released,0.0
2,16,Released,5.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Executing,37.0,3.0,4.0,0.0,0.0,31.0,50.0,2.0
Released,142.0,0.0,5.0,1.0,1.0,31.0,23.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Executing,37.0,3.0,4.0,0.0,0.0,31.0,50.0,2.0
1,Released,142.0,0.0,5.0,1.0,1.0,31.0,23.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Executing,37.0,3.0,4.0,0.0,0.0,31.0,50.0,2.0
1,Released,142.0,0.0,5.0,1.0,1.0,31.0,23.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()